In [ ]:
import numpy as np
import cmath

def rician_fading(K_db, size):
    """Generate Rician fading with given K-factor in dB."""
    K_linear = 10 ** (K_db / 10)  # Convert dB to linear
    los = np.sqrt(K_linear / (K_linear + 1))  # LOS component
    nlos = np.sqrt(1 / (K_linear + 1)) * (np.random.normal(0, 1, size) + 1j * np.random.normal(0, 1, size)) / np.sqrt(2)
    return los + nlos

def calculate_distance(coord1, coord2):
    """Calculate Euclidean distance between two 3D coordinates."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def generate_ris_channel_model(ref_path_loss, tx, rx, ris_pos, N_ris, p_block=0.3, p_attenuate=0.3, attenuate_db_range=(10, 20)):
    """
    Generate RIS-aided channel model with blockage, Rician fading, and attenuation.
    
    Args:
        ref_path_loss: Reference path loss at 1m.
        tx: Transmitter coordinates (x, y, z).
        rx: Receiver coordinates (x, y, z).
        ris_pos: RIS coordinates (x, y, z).
        N_ris: Number of RIS elements.
        p_block: Probability of direct path blockage (default: 0.3).
        p_attenuate: Probability of attenuation for non-blocked direct paths (default: 0.3).
        attenuate_db_range: Range of attenuation in dB for non-blocked paths (default: (10, 20)).
    
    Returns:
        h_d: Direct channel (1 x chan_realisations).
        g_BR: BS-RIS channel (N_ris x chan_realisations).
        g_RU: RIS-UE channel (N_ris x chan_realisations).
    """
    h_d = np.zeros((1, chan_realisations), dtype=complex)
    g_BR = np.zeros((N_ris, chan_realisations), dtype=complex)
    g_RU = np.zeros((N_ris, chan_realisations), dtype=complex)

    epsilon = 1e-10
    # Distances
    d_tx_rx = max(calculate_distance(tx, rx), epsilon)
    d_tx_ris = max(calculate_distance(tx, ris_pos), epsilon)
    d_ris_rx = max(calculate_distance(ris_pos, rx), epsilon)

    # Path loss (exponent 3 for urban environment)
    pl_direct = cmath.sqrt(ref_path_loss / (d_tx_rx ** 3))
    pl_BR = cmath.sqrt(ref_path_loss / (d_tx_ris ** 3))
    pl_RU = cmath.sqrt(ref_path_loss / (d_ris_rx ** 3))

    # Direct path: 30% blocked, 70% Rician with random K
    block_mask = np.random.rand(chan_realisations) < p_block  # True where blocked
    non_blocked = ~block_mask

    # Rician fading for direct path
    K_db = np.random.choice([0, 3, 7, 10], size=chan_realisations, p=[0.4, 0.3, 0.2, 0.1])
    fading_direct = np.zeros((1, chan_realisations), dtype=complex)
    for i in range(chan_realisations):
        if non_blocked[i]:
            fading_direct[0, i] = rician_fading(K_db[i], size=1)

    # Apply random attenuation to some non-blocked paths
    attenuate_mask = np.random.rand(chan_realisations) < p_attenuate  # True where attenuated
    attenuate_mask = attenuate_mask & non_blocked  # Only attenuate non-blocked paths
    attenuate_db = np.random.uniform(attenuate_db_range[0], attenuate_db_range[1], size=chan_realisations)
    attenuate_linear = 10 ** (-attenuate_db / 10)

    h_d[0, :] = fading_direct * pl_direct
    h_d[0, block_mask] = 0  # Block direct path
    h_d[0, attenuate_mask] *= attenuate_linear[attenuate_mask]  # Apply attenuation

    # Rician fading for RIS paths
    K_db_ris = np.random.choice([0, 3, 7, 10], size=chan_realisations, p=[0.4, 0.3, 0.2, 0.1])
    fading_BR = np.zeros((N_ris, chan_realisations), dtype=complex)
    fading_RU = np.zeros((N_ris, chan_realisations), dtype=complex)
    for i in range(chan_realisations):
        fading_BR[:, i] = rician_fading(K_db_ris[i], size=N_ris)
        fading_RU[:, i] = rician_fading(K_db_ris[i], size=N_ris)

    g_BR = fading_BR * pl_BR
    g_RU = fading_RU * pl_RU

    return h_d, g_BR, g_RU

In [ ]:
from functools import partial
from scipy.optimize import minimize

def generate_random_coordinates(n_devices, x_range, y_range, z_range):
    """Generate random coordinates for devices in a 3D space."""
    coordinates = np.zeros((n_devices, 3))
    for i in range(n_devices):
        x = np.random.uniform(*x_range)
        y = np.random.uniform(*y_range)
        z = np.random.uniform(*z_range)
        coordinates[i] = [x, y, z]
    return coordinates

def generate_ris_channel_gains(h_d, g_BR, g_RU, theta):
    """Calculate channel gains with RIS phase shifts."""
    N_ris = g_BR.shape[0]
    h_r = np.zeros((1, chan_realisations), dtype=complex)
    for n in range(N_ris):
        h_r += g_RU[n, :] * np.exp(1j * theta[n]) * g_BR[n, :]
    h_total = h_d + h_r
    g = np.abs(h_total) ** 2
    return g

def ris_spectral_efficiency(P, g, N0, W):
    """Calculate spectral efficiency for RIS-aided system."""
    epsilon = 1e-10
    term = np.maximum(1 + P * g / (N0 * W), epsilon)
    SE = np.log2(term)
    return np.mean(SE)

def ris_energy_efficiency(P, g, N0, W, Pc):
    """Calculate energy efficiency."""
    SE = ris_spectral_efficiency(P, g, N0, W)
    EE = SE / (P + Pc)
    return EE

def optimize_ris_phases(h_d, g_BR, g_RU, P, N0, W, Pc, objective='SE'):
    """Optimize RIS phase shifts."""
    N_ris = g_BR.shape[0]
    initial_theta = np.random.uniform(0, 2 * np.pi, N_ris)

    def objective_fn(theta):
        g = generate_ris_channel_gains(h_d, g_BR, g_RU, theta)
        if objective == 'SE':
            return -ris_spectral_efficiency(P, g, N0, W)
        else:
            return -ris_energy_efficiency(P, g, N0, W, Pc)

    bounds = [(0, 2 * np.pi)] * N_ris
    result = minimize(objective_fn, initial_theta, bounds=bounds, method='SLSQP')
    theta_opt = result.x
    g_opt = generate_ris_channel_gains(h_d, g_BR, g_RU, theta_opt)
    return theta_opt, g_opt

def process_ris_sample(sample, ref_path_loss, N0, W, Pc, P, N_ris, ris_pos):
    """Process a single sample for the dataset."""
    # Generate coordinates
    tx = generate_random_coordinates(1, x_range, y_range, z_range)[0]
    rx = generate_random_coordinates(1, x_range, y_range, z_range)[0]

    # Generate channels with blockage, Rician fading, and attenuation
    h_d, g_BR, g_RU = generate_ris_channel_model(ref_path_loss, tx, rx, ris_pos, N_ris, p_block=0.3, p_attenuate=0.3, attenuate_db_range=(10, 20))

    # Optimize for SE and EE
    theta_se, g_se = optimize_ris_phases(h_d, g_BR, g_RU, P, N0, W, Pc, objective='SE')
    theta_ee, g_ee = optimize_ris_phases(h_d, g_BR, g_RU, P, N0, W, Pc, objective='EE')

    # Normalize channel gains
    mean_g_se, std_g_se = np.mean(g_se), np.std(g_se)
    mean_g_ee, std_g_ee = np.mean(g_ee), np.std(g_ee)
    normalized_g_se = np.round(((g_se - mean_g_se) / std_g_se) * 100).astype(int) if std_g_se > 0 else np.zeros_like(g_se)
    normalized_g_ee = np.round(((g_ee - mean_g_ee) / std_g_ee) * 100).astype(int) if std_g_ee > 0 else np.zeros_like(g_ee)

    # Normalize phase shifts (map [0, 2π] to [0, 100])
    normalized_theta_se = np.round((theta_se / (2 * np.pi)) * 100).astype(int)
    normalized_theta_ee = np.round((theta_ee / (2 * np.pi)) * 100).astype(int)

    # Format output
    line_se = f"If channel gains are {', '.join(map(str, normalized_g_se[:4]))}, then RIS phase shifts are {', '.join(map(str, normalized_theta_se))}.\\n"
    line_ee = f"If channel gains are {', '.join(map(str, normalized_g_ee[:4]))}, then RIS phase shifts are {', '.join(map(str, normalized_theta_ee))}.\\n"

    return line_se, line_ee

# System parameters (from notebook)
noise_power_density = 5.0119e-21
W = 10e6
circuit_power = 1
max_power = 1
num_samples = 10000
chan_realisations = 100
N_ris = 32
x_range, y_range, z_range = (0, 1000), (0, 1000), (0, 50)
ris_position = [500, 500, 25]
p_0 = -0.001

# Generate dataset
np.random.seed(42)
process_sample_partial = partial(process_ris_sample, ref_path_loss=p_0, N0=noise_power_density, W=W, Pc=circuit_power, P=max_power, N_ris=N_ris, ris_pos=ris_position)
results_se, results_ee = zip(*map(process_sample_partial, range(num_samples)))

# Save to files
with open('ris_se.txt', 'w') as file_se:
    file_se.writelines(results_se)
with open('ris_ee.txt', 'w') as file_ee:
    file_ee.writelines(results_ee)